<a href="https://colab.research.google.com/github/fpsomad/fpsomad/blob/main/Darknet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# εισαγωγή βιβλιοθηκών
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
# εισαγωγή συνόλου δεδομένων
data = pd.read_csv('https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/DarkNet.csv')
data.info()



<ipython-input-2-4e88a7a3b017>:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/DarkNet.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68580 entries, 0 to 68579
Data columns (total 83 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Src_IP                      68580 non-null  object 
 1   Src_Port                    68580 non-null  int64  
 2   Dst_IP                      68580 non-null  object 
 3   Dst_Port                    68580 non-null  int64  
 4   Protocol                    68580 non-null  int64  
 5   Flow_Duration               68580 non-null  int64  
 6   Total_Fwd_Packet            68580 non-null  int64  
 7   Total_Bwd_packets           68580 non-null  int64  
 8   Total_Length_of_Fwd_Packet  68580 non-null  int64  
 9   Total_Length_of_Bwd_Packet  68580 non-null  int64  
 10  Fwd_Packet_Length_Max       68580 non-null  int64  
 11  Fwd_Packet_Length_Min       68580 non-null  int64  
 12  Fwd_Packet_Length_Mean      68580 non-null  float64
 13  Fwd_Packet_Length_Std       685

In [4]:
# ενοποίηση ονομασιών υπηρεσιών DarkNet
data['Label-2'] = data['Label-2'].replace(['AUDIO-STREAMING', 'Video-streaming', 'File-transfer'], ['Audio-Streaming', 'Video-Streaming','File-Transfer'])
data['Label-2'].unique()
data.groupby(data['Label-2']).size()

Label-2
Audio-Streaming     7539
Browsing            5192
Chat               11478
Email               6145
File-Transfer      11182
P2P                13711
VOIP                3566
Video-Streaming     9767
dtype: int64

In [ ]:
# ορισμός κλάσης για ταξινόμηση κίνησης "Class: 0 (normal)  1 (DarkNet)"


In [ ]:
# ταξινόμηση δεδομένων με βάση την κίνηση
data.groupby(data['Label-1']).size()


Label-1
Non-Tor    30931
NonVPN     22740
Tor         1392
VPN        13517
dtype: int64

In [ ]:
# ταξινόμηση δεδομένων με βάση την εφαρμογή
data.groupby(data['Label-2']).size()


Label-2
AUDIO-STREAMING     1484
Audio-Streaming     6055
Browsing            5192
Chat               11478
Email               6145
File-Transfer      11098
File-transfer         84
P2P                13711
VOIP                3566
Video-Streaming     9486
Video-streaming      281
dtype: int64